# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
df = pd.read_csv("../output_data/cities.csv")
del df["Unnamed: 0"]
df

,CITY,LAT,LNG,TEMP,MAXTEMP,HUMIDITY,CLOUDINESS,WINDSPEED
0,Richards Bay,-28.7830,32.0377,70.39,70.39,61,0,14.92
1,Punta Arenas,-53.1500,-70.9167,42.03,42.03,72,0,39.12
2,Hilo,19.7297,-155.0900,72.25,72.25,70,0,6.91
3,Salalah,17.0151,54.0924,77.09,77.09,88,75,5.75
4,Kahului,20.8947,-156.4700,71.62,71.62,68,20,14.00
5,Arroio Grande,-32.2375,-53.0869,56.19,56.19,64,83,13.15
6,Bredasdorp,-34.5322,20.0403,50.85,50.85,72,100,11.27
7,Mataura,-46.1927,168.8643,32.56,32.56,97,67,2.30
8,Albany,42.6001,-73.9662,84.33,84.33,40,52,1.99
9,Kutum,14.2000,24.6667,79.38,79.38,55,100,8.08


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key) 

In [5]:
locations = df[["LAT", "LNG"]]

weights = df["HUMIDITY"].astype(float)

In [6]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [7]:
# best_cities_df = df.loc[ df["TEMP"]]

,CITY,LAT,LNG,TEMP,MAXTEMP,HUMIDITY,CLOUDINESS,WINDSPEED
0,Richards Bay,-28.7830,32.0377,70.39,70.39,61,0,14.92
1,Punta Arenas,-53.1500,-70.9167,42.03,42.03,72,0,39.12
2,Hilo,19.7297,-155.0900,72.25,72.25,70,0,6.91
3,Salalah,17.0151,54.0924,77.09,77.09,88,75,5.75
4,Kahului,20.8947,-156.4700,71.62,71.62,68,20,14.00
5,Arroio Grande,-32.2375,-53.0869,56.19,56.19,64,83,13.15
6,Bredasdorp,-34.5322,20.0403,50.85,50.85,72,100,11.27
7,Mataura,-46.1927,168.8643,32.56,32.56,97,67,2.30
8,Albany,42.6001,-73.9662,84.33,84.33,40,52,1.99
9,Kutum,14.2000,24.6667,79.38,79.38,55,100,8.08


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
